# What borough has the fastest NYC taxi drivers?

In [1]:
from __future__ import division

In [2]:
%reload_ext autotime

In [3]:
from blaze import by, compute, transform, Data, sin, cos, atan2, sqrt, radians, summary, greatest, symbol
from odo import odo, drop, resource, Temp, CSV, S3

time: 2.38 s


In [4]:
connect_args = dict(sslmode='verify-ca')

time: 851 µs


In [5]:
d = Data('redshift://cio@localhost:15440/dev::trip', connect_args=connect_args)

time: 481 ms


In [6]:
d

medallion                      hack_license  \
0   DFD2202EE08F7A8DC9A57B02ACB81FE2  51EE87E3205C985EF8431D850C786310   
1   0B57B9633A2FECD3D3B1944AFC7471CF  CCD4367B417ED6634D986F573A552A62   
2   E12F6AF991172EAC3553144A0AF75A19  06918214E951FA0003D1CC54955C2AB0   
3   3349F919AA8AE5DC9C50A3773EA45BD8  7CE849FEF67514F080AF80D990F7EF7F   
4   0C5296F3C8B16E702F8F2E06F5106552  D2363240A9295EF570FC6069BC4F4C92   
5   4C005EEBAA7BF26B84B21586332488A2  351BE7D984BE17DB2FA80A748E816472   
6   A3B17384165197E18CA0A1BB61277EE9  B8396B62883EA332EFD2771A6B031D05   
7   0F9E0728AB1E40D5CEB0C6EDBF805CCB  8434E8A33D8C0150573FAA00B8A9ABF5   
8   2C159C8FCCDE50174CF6CFC07E75F1BA  51C567531541EE45C5EE86E956E46E4C   
9   DD98E2C3AF5C47B4449F720ECC5778D4  79807332B275653A2473554C7328500A   
10  E8950C1BF2CE598E95DE0CA5C89FA4BA  E4D2F480F3C91138B7BA9F1BDE710434   

   vendor_id  rate_code store_and_fwd_flag     pickup_datetime  \
0        CMT          1                  N 2013-01-07 23:54:15   
1        CMT          1                  N 2013-01-07 12:39:18   
2        CMT          1                  N 2013-01-08 09:50:05   
3        CMT          1                  N 2013-01-10 14:27:28   
4        CMT          1                  N 2013-01-07 22:25:46   
5        CMT          1                  N 2013-01-07 21:13:02   
6        CMT          1                  N 2013-01-07 06:26:32   
7        CMT          1                  N 2013-01-05 19:04:43   
8        CMT          1                  N 2013-01-07 04:45:14   
9        CMT          1                  N 2013-01-02 06:58:08   
10       CMT          1                  N 2013-01-05 21:23:06   

      dropoff_datetime  passenger_count  trip_time_in_secs  trip_distance  \
0  2013-01-07 23:58:20                2                244            0.7   
1  2013-01-07 13:10:56                3               1898           10.7   
2  2013-01-08 10:02:54                1                768            0.7   
3  2013-01-10 14:45:21                1               1073            4.4   
4  2013-01-07 22:36:56                1                669            2.3   
5  2013-01-07 21:22:31                1                568            1.1   
6  2013-01-07 06:28:24                2                111            0.5   
7  2013-01-05 19:13:58                1                555            2.8   
8  2013-01-07 04:48:56                1                222            0.9   
9  2013-01-02 07:24:24                1               1576            3.0   
10 2013-01-05 21:25:19                1                132            0.3   

      ...     pickup_latitude  dropoff_longitude  dropoff_latitude  \
0     ...           40.759945         -73.984734         40.759388   
1     ...           40.756775         -73.865250         40.770630   
2     ...           40.749294         -73.988686         40.759052   
3     ...           40.708702         -73.987846         40.756104   
4     ...           40.756714         -73.977615         40.787575   
5     ...           40.764679         -73.977684         40.777004   
6     ...           40.769375         -73.967133         40.763699   
7     ...           40.761139         -73.938515         40.792332   
8     ...           40.749622         -73.978531         40.751308   
9     ...           40.759373         -73.862022         40.768017   
10    ...           40.779442         -73.960724         40.775745   

    tolls_amount  tip_amount  total_amount  mta_tax  fare_amount  \
0            0.0           0           6.0      0.5          5.0   
1            4.8           0          39.3      0.5         34.0   
2            0.0           0           9.5      0.5          9.0   
3            0.0           0          17.0      0.5         16.5   
4            0.0           0          11.5      0.5         10.5   
5            0.0           0           9.0      0.5          8.0   
6            0.0           0           5.0      0.5          4.0   
7            0.0           0       

time: 2.13 s


In [7]:
def haversine_distance(start, stop, R=3959):
    """Compute the distance between two sets of `start` and `stop` lat, lon points
    """
    # http://andrew.hedges.name/experiments/haversine/
    lat1, lon1 = start
    lat2, lon2 = stop
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(radians(dlat) / 2.0) ** 2 + cos(lat1) * cos(lat2) * sin(radians(dlon) / 2.0) ** 2
    return R * 2 * atan2(sqrt(greatest(a, 0.0)), sqrt(greatest(1.0 - a, 0.0)))

time: 4.15 ms


In [8]:
max_distance = haversine_distance(start=[40.477399, -74.259090], stop=[40.917577, -73.700272])
max_distance

48.11056468852813

time: 2.32 ms


## Reduce the data to the area defined as NYC

In [9]:
# http://hafen.github.io/taxi/#reading-in-to-r

min_lat, max_lat = 40.477399, 40.917577
min_lon, max_lon = -74.259090, -73.700272

valid = d[
  (d.trip_distance > 0) &
  (d.trip_distance <= max_distance) &
  (d.trip_time_in_secs > 0) &
  (d.passenger_count > 0) & (d.passenger_count <= 5) &
  (d.pickup_latitude >= min_lat) & (d.pickup_latitude <= max_lat) &
  (d.dropoff_latitude >= min_lat) & (d.dropoff_latitude <= max_lat) &
  (d.pickup_longitude >= min_lon) & (d.pickup_longitude <= max_lon) &
  (d.dropoff_longitude >= min_lon) & (d.dropoff_longitude <= max_lon)
]

time: 161 ms


## Add a column containing the average speed in MPH

In [10]:
computed_distance = haversine_distance(start=[valid.pickup_latitude, valid.pickup_longitude],
                                       stop=[valid.dropoff_latitude, valid.dropoff_longitude])
trip_time_in_hours = valid.trip_time_in_secs.coerce('float64') / 3600.0
calcd = transform(valid,
                  avg_speed_in_mph=valid.trip_distance / trip_time_in_hours,
                  trip_time_in_hours=trip_time_in_hours)

time: 338 ms


In [11]:
calcd

medallion                      hack_license  \
0   DFD2202EE08F7A8DC9A57B02ACB81FE2  51EE87E3205C985EF8431D850C786310   
1   0B57B9633A2FECD3D3B1944AFC7471CF  CCD4367B417ED6634D986F573A552A62   
2   E12F6AF991172EAC3553144A0AF75A19  06918214E951FA0003D1CC54955C2AB0   
3   3349F919AA8AE5DC9C50A3773EA45BD8  7CE849FEF67514F080AF80D990F7EF7F   
4   0C5296F3C8B16E702F8F2E06F5106552  D2363240A9295EF570FC6069BC4F4C92   
5   4C005EEBAA7BF26B84B21586332488A2  351BE7D984BE17DB2FA80A748E816472   
6   A3B17384165197E18CA0A1BB61277EE9  B8396B62883EA332EFD2771A6B031D05   
7   0F9E0728AB1E40D5CEB0C6EDBF805CCB  8434E8A33D8C0150573FAA00B8A9ABF5   
8   2C159C8FCCDE50174CF6CFC07E75F1BA  51C567531541EE45C5EE86E956E46E4C   
9   DD98E2C3AF5C47B4449F720ECC5778D4  79807332B275653A2473554C7328500A   
10  E8950C1BF2CE598E95DE0CA5C89FA4BA  E4D2F480F3C91138B7BA9F1BDE710434   

   vendor_id  rate_code store_and_fwd_flag     pickup_datetime  \
0        CMT          1                  N 2013-01-07 23:54:15   
1        CMT          1                  N 2013-01-07 12:39:18   
2        CMT          1                  N 2013-01-08 09:50:05   
3        CMT          1                  N 2013-01-10 14:27:28   
4        CMT          1                  N 2013-01-07 22:25:46   
5        CMT          1                  N 2013-01-07 21:13:02   
6        CMT          1                  N 2013-01-07 06:26:32   
7        CMT          1                  N 2013-01-05 19:04:43   
8        CMT          1                  N 2013-01-07 04:45:14   
9        CMT          1                  N 2013-01-02 06:58:08   
10       CMT          1                  N 2013-01-05 21:23:06   

      dropoff_datetime  passenger_count  trip_time_in_secs  trip_distance  \
0  2013-01-07 23:58:20                2                244            0.7   
1  2013-01-07 13:10:56                3               1898           10.7   
2  2013-01-08 10:02:54                1                768            0.7   
3  2013-01-10 14:45:21                1               1073            4.4   
4  2013-01-07 22:36:56                1                669            2.3   
5  2013-01-07 21:22:31                1                568            1.1   
6  2013-01-07 06:28:24                2                111            0.5   
7  2013-01-05 19:13:58                1                555            2.8   
8  2013-01-07 04:48:56                1                222            0.9   
9  2013-01-02 07:24:24                1               1576            3.0   
10 2013-01-05 21:25:19                1                132            0.3   

          ...          dropoff_latitude  tolls_amount  tip_amount  \
0         ...                 40.759388           0.0           0   
1         ...                 40.770630           4.8           0   
2         ...                 40.759052           0.0           0   
3         ...                 40.756104           0.0           0   
4         ...                 40.787575           0.0           0   
5         ...                 40.777004           0.0           0   
6         ...                 40.763699           0.0           0   
7         ...                 40.792332           0.0           0   
8         ...                 40.751308           0.0           0   
9         ...                 40.768017           4.8           0   
10        ...                 40.775745           0.0           0   

    total_amount  mta_tax  fare_amount  payment_type  surcharge  \
0            6.0      0.5          5.0           CSH        0.5   
1           39.3      0.5         34.0           CSH        0.0   
2            9.5      0.5          9.0           CSH        0.0   
3           17.0      0.5         16.5           CSH        0.0   
4           11.5      0.5         10.5           CSH        0.5   
5            9.0      0.5          8.0           CSH        0.5   
6            5.0      0.5          4.0           CSH        0.5   
7           10.5      0.5         10.0           CSH   

time: 2.65 s


### Assume that > 120 MPH is invalid

In [12]:
data = calcd[(calcd.avg_speed_in_mph > 5) & (calcd.avg_speed_in_mph <= 120)][
    [
        'avg_speed_in_mph', 
        'pickup_latitude', 
        'pickup_longitude', 
        'dropoff_latitude', 
        'dropoff_longitude'
    ]
]

time: 57.5 ms


In [13]:
data

avg_speed_in_mph  pickup_latitude  pickup_longitude  dropoff_latitude  \
0           9.424084        40.757977        -73.978165         40.751171   
1          13.500000        40.748528        -73.976250         40.747868   
2           9.632107        40.743137        -73.980072         40.735336   
3          13.354839        40.791321        -73.971497         40.775921   
4          10.355330        40.756287        -73.973732         40.756832   
5          38.414634        40.648766        -73.783310         40.748207   
6          13.740458        40.749954        -73.992172         40.744553   
7          23.890696        40.769142        -73.862709         40.762295   
8          21.224490        40.776482        -73.948334         40.733841   
9          19.200000        40.752102        -73.982384         40.736393   
10         17.234043        40.743130        -73.991173         40.702297   

    dropoff_longitude  
0          -73.989838  
1          -74.002586  
2          -73.982712  
3          -73.964478  
4          -73.998413  
5          -73.988914  
6          -73.996750  
7          -73.982079  
8          -73.991760  
9          -73.993332  
...

time: 2.52 s


In [15]:
print(compute(data))

SELECT trip.trip_distance / (CAST(trip.trip_time_in_secs AS FLOAT(53)) / %(param_1)s) AS avg_speed_in_mph, trip.pickup_latitude, trip.pickup_longitude, trip.dropoff_latitude, trip.dropoff_longitude 
FROM trip 
WHERE trip.trip_distance > %(trip_distance_1)s AND trip.trip_distance <= %(trip_distance_2)s AND trip.trip_time_in_secs > %(trip_time_in_secs_1)s AND trip.passenger_count > %(passenger_count_1)s AND trip.passenger_count <= %(passenger_count_2)s AND trip.pickup_latitude >= %(pickup_latitude_1)s AND trip.pickup_latitude <= %(pickup_latitude_2)s AND trip.dropoff_latitude >= %(dropoff_latitude_1)s AND trip.dropoff_latitude <= %(dropoff_latitude_2)s AND trip.pickup_longitude >= %(pickup_longitude_1)s AND trip.pickup_longitude <= %(pickup_longitude_2)s AND trip.dropoff_longitude >= %(dropoff_longitude_1)s AND trip.dropoff_longitude <= %(dropoff_longitude_2)s AND trip.trip_distance / (CAST(trip.trip_time_in_secs AS FLOAT(53)) / %(param_2)s) > %(param_3)s AND trip.trip_distance / (CAST(t

## A small aside, with `odo`

### Let's look at the distribution of rides by borough

#### Q: Where do we get borough geolocation data?
#### A: From NYC's own API!

In [16]:
import requests as r

time: 717 µs


In [17]:
resp = r.get(
    'https://data.cityofnewyork.us/api/geospatial/tqmj-j8zm',
    params=dict(method='export', format='KML')
)

time: 3.74 s


In [18]:
import lxml
import lxml.etree

time: 28.6 ms


#### Parse the KML with `lxml`

In [19]:
kml = lxml.etree.fromstring(resp.text.encode('utf8'))

time: 9.32 ms


In [20]:
print(resp.text)

<?xml version="1.0" encoding="UTF-8"?><kml xmlns="http://www.opengis.net/kml/2.2" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/kml/2.2 http://schemas.opengis.net/kml/2.2.0/ogckml22.xsd">
   <Document xmlns:atom="http://purl.org/atom/ns#">
      <name>geo_tqmj-j8zm:geo_tqmj-j8zm-1</name>
      <LookAt>
         <longitude>-73.97867494308187</longitude>
         <latitude>40.70615701593643</latitude>
         <altitude>81791.8316066984</altitude>
         <range>66092.09005639808</range>
         <tilt>0.0</tilt>
         <heading>0.0</heading>
         <altitudeMode>clampToGround</altitudeMode>
      </LookAt>
      <Placemark id="geo_tqmj-j8zm-1.1">
         <name><![CDATA[geo_tqmj-j8zm-1.1]]></name>
         <description><![CDATA[<h4>geo_tqmj-j8zm-1</h4>

<ul class="textattributes">
  
  <li><strong><span class="atr-name">BoroCode</span>:</strong> <span class="atr-value">5</span></li>
  <li><strong><span class="atr-name">BoroName</sp

#### `Placemark` elements delineate boroughs

In [21]:
placemarks = kml.xpath("//*[local-name()='Placemark']")

time: 2.24 ms


#### Create some functions to pull out the borough names and points

In [22]:
from toolz.curried import map
from toolz import pipe, concat

import io


def find_borough_name(text):
    return re.search(r'.*(Brooklyn|Manhattan|Bronx|Staten Island|Queens).*', text).group(1)


def parse_coords(p, name):
    coords = p.xpath('.//*[local-name()="coordinates"]/text()')
    return pipe(coords,
                map(str.split),
                map(lambda x: '\n'.join(map(lambda y: '%s,%s' % (y, name), x))),
                '\n'.join)


def parse_kml(placemarks):
    result = []
    
    for p in placemarks:
        desc, = p.xpath(".//*[local-name()='description']")
        name = find_borough_name(desc.text)        
        result.append(parse_coords(p, name))
    return '\n'.join(result)

time: 8.58 ms


#### Parse the location data

In [23]:
geodata = parse_kml(placemarks)
geodata[:geodata.find('\n')]

'-74.15340460608331,40.58083062671828,Staten Island'

time: 45.4 ms


#### Inspect our data with pandas

In [24]:
df = pd.read_csv(io.StringIO(geodata), names=['lon', 'lat', 'name']).drop_duplicates().reset_index(drop=True)

time: 70.6 ms


In [25]:
df.head()

lon        lat           name
0 -74.153405  40.580831  Staten Island
1 -74.050500  40.566400  Staten Island
2 -74.050300  40.566400  Staten Island
3 -74.050100  40.566400  Staten Island
4 -74.050000  40.566400  Staten Island

time: 5.69 ms


#### After we're satisfied, let's throw everything into redshift for later analysis

In [26]:
boros = odo(
    odo(df, Temp(S3(CSV))),
    'redshift://cio@localhost:15440/dev::boros',
    connect_args=connect_args
)

time: 15.9 s


#### Compute the bounding box for each borough

In [27]:
bounds = sa.select([
    boros.c.name,
    sa.func.box(sa.func.point(sa.func.min(boros.c.lon),
                              sa.func.min(boros.c.lat)),
                sa.func.point(sa.func.max(boros.c.lon),
                              sa.func.max(boros.c.lat))).label('boro_box')
]).group_by(boros.c.name).alias()

time: 3.89 ms


#### Get out the SQLAlchemy table when blaze isn't enough

In [28]:
data

avg_speed_in_mph  pickup_latitude  pickup_longitude  dropoff_latitude  \
0          20.849421        40.731781        -74.006683         40.750660   
1           9.444444        40.764252        -73.966743         40.743763   
2           9.404389        40.786983        -73.977936         40.806370   
3          13.529412        40.774307        -73.988510         40.755325   
4          15.949367        40.767704        -73.968925         40.776566   
5          12.504570        40.771278        -73.956505         40.732460   
6          15.882353        40.689701        -73.992111         40.679295   
7           5.343228        40.748310        -73.977127         40.751053   
8          11.206226        40.724407        -73.993202         40.748741   
9          18.276923        40.721008        -73.993828         40.723228   
10         12.129380        40.750023        -73.991150         40.741150   

    dropoff_longitude  
0          -73.994499  
1          -73.983322  
2          -73.952919  
3          -73.981094  
4          -73.961990  
5          -73.996368  
6          -74.007156  
7          -73.990913  
8          -73.984833  
9          -73.952400  
...

time: 1.69 s


In [29]:
nyc = compute(data).alias()

time: 247 ms


In [30]:
joined = nyc.join(
    bounds,
    onclause=sa.func.lseg(
        sa.func.point(nyc.c.pickup_longitude, nyc.c.pickup_latitude),
        sa.func.point(nyc.c.dropoff_longitude, nyc.c.dropoff_latitude)
    ).op('@')(bounds.c.boro_box)
)

time: 2.61 ms


In [31]:
sel = sa.select([
    bounds.c.name,
    nyc.c.avg_speed_in_mph, 
    nyc.c.pickup_latitude,
    nyc.c.pickup_longitude,
    nyc.c.dropoff_latitude,
    nyc.c.dropoff_longitude
]).select_from(joined).alias()

time: 1.91 ms


In [32]:
per_boro_speeds = sa.select([
    sel.c.name,
    sa.func.avg(sel.c.avg_speed_in_mph).label('avg_speed'),
    sa.func.count(sel.c.avg_speed_in_mph).label('nsamples')
]).group_by(sel.c.name)

time: 1.95 ms


In [33]:
import sqlparse

time: 14 ms


In [34]:
print(sqlparse.format(str(per_boro_speeds), reindent=True))

SELECT anon_1.name,
       avg(anon_1.avg_speed_in_mph) AS avg_speed,
       count(anon_1.avg_speed_in_mph) AS nsamples
FROM
  (SELECT anon_2.name AS name,
          anon_3.avg_speed_in_mph AS avg_speed_in_mph,
          anon_3.pickup_latitude AS pickup_latitude,
          anon_3.pickup_longitude AS pickup_longitude,
          anon_3.dropoff_latitude AS dropoff_latitude,
          anon_3.dropoff_longitude AS dropoff_longitude
   FROM
     (SELECT trip.trip_distance / (CAST(trip.trip_time_in_secs AS FLOAT(53)) / %(param_1)s) AS avg_speed_in_mph,
             trip.pickup_latitude AS pickup_latitude,
             trip.pickup_longitude AS pickup_longitude,
             trip.dropoff_latitude AS dropoff_latitude,
             trip.dropoff_longitude AS dropoff_longitude
      FROM trip
      WHERE trip.trip_distance > %(trip_distance_1)s
        AND trip.trip_distance <= %(trip_distance_2)s
        AND trip.trip_time_in_secs > %(trip_time_in_secs_1)s
        AND trip.passenger_count > %(passe

In [35]:
speeds = odo(
    per_boro_speeds.order_by(sa.desc(per_boro_speeds.c.avg_speed)),
    pd.DataFrame,
    connect_args=connect_args
)

DatabaseError: (psycopg2.DatabaseError) SSL SYSCALL error: EOF detected
 [SQL: 'SELECT anon_1.name, avg(anon_1.avg_speed_in_mph) AS avg_speed, count(anon_1.avg_speed_in_mph) AS nsamples \nFROM (SELECT anon_2.name AS name, anon_3.avg_speed_in_mph AS avg_speed_in_mph, anon_3.pickup_latitude AS pickup_latitude, anon_3.pickup_longitude AS pickup_longitude, anon_3.dropoff_latitude AS dropoff_latitude, anon_3.dropoff_longitude AS dropoff_longitude \nFROM (SELECT trip.trip_distance / (CAST(trip.trip_time_in_secs AS FLOAT(53)) / %(param_1)s) AS avg_speed_in_mph, trip.pickup_latitude AS pickup_latitude, trip.pickup_longitude AS pickup_longitude, trip.dropoff_latitude AS dropoff_latitude, trip.dropoff_longitude AS dropoff_longitude \nFROM trip \nWHERE trip.trip_distance > %(trip_distance_1)s AND trip.trip_distance <= %(trip_distance_2)s AND trip.trip_time_in_secs > %(trip_time_in_secs_1)s AND trip.passenger_count > %(passenger_count_1)s AND trip.passenger_count <= %(passenger_count_2)s AND trip.

time: 15.1 s


In [ ]:
speeds